In [43]:
import numpy as np
from astropy.io import fits
import cv2
import os
import math

In [44]:
def min_max(x, ax, ay):
    min_x = x.min()
    max_x = x.max()
    a = np.zeros((ax, ay))
    b = np.zeros((ax, ay))
    z = np.zeros((ax, ay))
    for i in range(ax):
        for j in range(ay):
            z[i][j] = (x[i][j] - min_x)/(max_x - min_x) * 255
            a[i][j],b[i][j] = math.modf(z[i][j])

    return a,b

In [46]:
def image_conv(image):
    i = fits.getdata(image)
    cv2.imwrite("test.tif",i.astype('uint16'))
    c = cv2.imread("test.tif",-1)

    return c

In [52]:
# FITS画像ファイルのリストを作成
# image_files = ["../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491939184_b.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491942076_v.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491945018_w.fit",
#                  "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491947926_p.fit"]  # すべての画像ファイルを含むリスト

# image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489428266_b.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489432123_v.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489436012_w.fit",
#                  "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489439885_p.fit"]

image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490234689_b.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490238562_v.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490242451_w.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490246307_p.fit"]


# 出力ディレクトリとファイル名のプレフィックスを指定
home_dir = os.path.expanduser("~")
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_directory_op/")
file_prefix = 'registered_image_'  # ファイル名のプレフィックスを指定

hdulist = fits.open(image_files[2])  
hdu = hdulist[0]
data = hdu.data
header = hdu.header
x_axis = header["NAXIS1"]
y_axis = header["NAXIS2"]

name_prefix = []
for i,name in enumerate(image_files):
    parts = name.split(".f")
    part = parts[0].split("/data/")
    par = part[1].split("/st")
    p = f"{par[0]}{par[1]}_reg_"
    name_prefix.append(p)

name = image_files[0]
parts = name.split(".f")
part = parts[0].split("/data/")
par = part[1].split("/st")
p = f"{par[0]}"
name_prefix.append(p)

# 最初の画像を基準画像として読み込み
images = []
#少数部分の保存
float_images = []
f = []
images_uint16 = []
for i,filename in enumerate(image_files):
    images_uint16.append(image_conv(filename))
    f.append(fits.getdata(filename))
    a,b = min_max(f[i], x_axis, y_axis)
    if i == 2:
      reference_image = np.uint8(b)
    images.append(np.uint8(b))
    float_images.append(a)
  


In [51]:
for i in range(4):
    print((fits.getdata(image_files[i]) == images[i]).all())

True
True
True
True


In [55]:
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_perspective_test/")
registered_images = []


# 基準画像から他の画像への変換行列を計算し、変換を適用
for i,image_file in enumerate(images):
    
    # 画像の特徴点を検出
    akaze = cv2.AKAZE_create()
    keypoints1, descriptors1 = akaze.detectAndCompute(reference_image, None)
    keypoints2, descriptors2 = akaze.detectAndCompute(image_file, None)
    
    # 特徴点マッチングと変換行列の計算
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors2, descriptors1, k = 2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append([m])
    
    #適切なキーポイントを選択
    src_pts = np.float32([keypoints2[m[0].queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints1[m[0].trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, status = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    
    # 変換行列を適用して画像を基準画像の座標系に変換
    registered_image = cv2.warpPerspective(images_uint16[i], H, (image_file.shape[1], image_file.shape[0]))
    
    registered_images.append(registered_image)

for i in range(len(registered_images)):
    output_jpgname = f"{output_directory}{name_prefix[i]}.png"
    output_filename = f"{output_directory}{name_prefix[i]}.fits"
    cv2.imwrite(output_jpgname, registered_images[i])
    fits.writeto(output_filename, registered_images[i], overwrite=True)

In [36]:
image = "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490234689_b.fit"


i = fits.getdata(image)
io.imsave("test.tif",i.astype('uint16'))
c = cv2.imread("test.tif",-1)


/var/folders/pz/yjmbqcmx2t5cm5ff1q7nkqnc0000gn/T/ipykernel_12582/220205332.py:5: UserWarning: test.tif is a low contrast image
  io.imsave("test.tif",i.astype('uint16'))


In [25]:
type(c)

numpy.ndarray

In [37]:
print(c.dtype)

uint16


In [39]:
i

array([[3, 1, 4, ..., 1, 3, 2],
       [1, 3, 1, ..., 4, 4, 1],
       [5, 6, 2, ..., 4, 1, 6],
       ...,
       [4, 3, 4, ..., 2, 1, 6],
       [1, 1, 5, ..., 3, 3, 1],
       [1, 6, 1, ..., 4, 1, 2]], dtype=int16)

In [38]:
c

array([[3, 1, 4, ..., 1, 3, 2],
       [1, 3, 1, ..., 4, 4, 1],
       [5, 6, 2, ..., 4, 1, 6],
       ...,
       [4, 3, 4, ..., 2, 1, 6],
       [1, 1, 5, ..., 3, 3, 1],
       [1, 6, 1, ..., 4, 1, 2]], dtype=uint16)

In [41]:
print((i == c).all())

True


In [42]:
fits.writeto("test.fits", c, overwrite=True)